# 목차
## &nbsp;&nbsp;&nbsp;&nbsp;1. Pre-processing for movie-lens data (ml-latest-small, 2.4M)
## &nbsp;&nbsp;&nbsp;&nbsp;2. User-based (cosine)
## &nbsp;&nbsp;&nbsp;&nbsp;3. Item-based
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3-1. Item-based (cosine)
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3-2. Item-based (adjusted cosine)
## &nbsp;&nbsp;&nbsp;&nbsp;4. SVD

## ● 1번은 전처리 과정이므로 각각의 알고리즘 수행을 위해 아래를 참고한다.
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;User-based CF 돌리기 위해서 1번을 수행한 후 2번의 과정을 수행 ( 1 -> 2 )
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Item-based CF 돌리기 위해서 1번을 수행한 후 3번의 과정을 수행 ( 1 -> 3 )
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SVD 돌리기 위해서 1번을 수행한 후 4번의 과정을 수행 ( 1 -> 4 )
## =====================================================================

# 1. pre-processing for movie-lens data (ml-latest-small, 2.4M)

## ● 기본 데이터셋 생성

In [21]:
import pandas as pd
import numpy as np

In [22]:
file_path_movie = '/Users/morulabs/dev/source/git_hub/recsys/movie-lens/ml-latest-small/movies.csv'
file_path_rating = '/Users/morulabs/dev/source/git_hub/recsys/movie-lens/ml-latest-small/ratings.csv'

In [23]:
header_movie = ['item_id', 'title', 'genres']
df_movies = pd.read_csv(file_path_movie, header=0, names=header_movie)
df_movies

item_id                                              title  \
0           1                                   Toy Story (1995)   
1           2                                     Jumanji (1995)   
2           3                            Grumpier Old Men (1995)   
3           4                           Waiting to Exhale (1995)   
4           5                 Father of the Bride Part II (1995)   
5           6                                        Heat (1995)   
6           7                                     Sabrina (1995)   
7           8                                Tom and Huck (1995)   
8           9                                Sudden Death (1995)   
9          10                                   GoldenEye (1995)   
10         11                     American President, The (1995)   
11         12                 Dracula: Dead and Loving It (1995)   
12         13                                       Balto (1995)   
13         14                                       Nixon (1995)   
14         15                            Cutthroat Island (1995)   
15         16                                      Casino (1995)   
16         17                       Sense and Sensibility (1995)   
17         18                                  Four Rooms (1995)   
18         19              Ace Ventura: When Nature Calls (1995)   
19         20                                 Money Train (1995)   
20         21                                  Get Shorty (1995)   
21         22                                     Copycat (1995)   
22         23                                   Assassins (1995)   
23         24                                      Powder (1995)   
24         25                           Leaving Las Vegas (1995)   
25         26                                     Othello (1995)   
26         27                                Now and Then (1995)   
27         28                                  Persuasion (1995)   
28         29  City of Lost Children, The (Cité des enfants p...   
29         30  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
...       ...                                                ...   
9095   159690  Teenage Mutant Ninja Turtles: Out of the Shado...   
9096   159755          Popstar: Never Stop Never Stopping (2016)   
9097   159858                             The Conjuring 2 (2016)   
9098   159972                     Approaching the Unknown (2016)   
9099   160080                                Ghostbusters (2016)   
9100   160271                        Central Intelligence (2016)   
9101   160438                                Jason Bourne (2016)   
9102   160440                             The Maid's Room (2014)   
9103   160563                        The Legend of Tarzan (2016)   
9104   160565                    The Purge: Election Year (2016)   
9105   160567              Mike & Dave Need Wedding Dates (2016)   
9106   160590                         Survive and Advance (2013)   
9107   160656                                    Tallulah (2016)   
9108   160718                                       Piper (2016)   
9109   160954                                       Nerve (2016)   
9110   161084                       My Friend Rockefeller (2015)   
9111   161155                                   Sunspring (2016)   
9112   161336                  Author: The JT LeRoy Story (2016)   
9113   161582                          Hell or High Water (2016)   
9114   161594               Kingsglaive: Final Fantasy XV (2016)   
9115   161830                                        Body (2015)   
9116   161918                Sharknado 4: The 4th Awakens (2016)   
9117   161944              The Last Brickmaker in America (2001)   
9118   162376                                    Stranger Things   
9119   162542                                      Rustom (2016)   
9120   162672                                Mohenjo Daro (2016)   
9121   163056                               Shin Godzilla (201

In [24]:
header_rating = ['user_id', 'item_id', 'rating', 'timestamp']
df_ratings = pd.read_csv(file_path_rating, header=0, names=header_rating)
df_ratings

user_id  item_id  rating   timestamp
0             1       31     2.5  1260759144
1             1     1029     3.0  1260759179
2             1     1061     3.0  1260759182
3             1     1129     2.0  1260759185
4             1     1172     4.0  1260759205
5             1     1263     2.0  1260759151
6             1     1287     2.0  1260759187
7             1     1293     2.0  1260759148
8             1     1339     3.5  1260759125
9             1     1343     2.0  1260759131
10            1     1371     2.5  1260759135
11            1     1405     1.0  1260759203
12            1     1953     4.0  1260759191
13            1     2105     4.0  1260759139
14            1     2150     3.0  1260759194
15            1     2193     2.0  1260759198
16            1     2294     2.0  1260759108
17            1     2455     2.5  1260759113
18            1     2968     1.0  1260759200
19            1     3671     3.0  1260759117
20            2       10     4.0   835355493
21            2       17     5.0   835355681
22            2       39     5.0   835355604
23            2       47     4.0   835355552
24            2       50     4.0   835355586
25            2       52     3.0   835356031
26            2       62     3.0   835355749
27            2      110     4.0   835355532
28            2      144     3.0   835356016
29            2      150     5.0   835355395
...         ...      ...     ...         ...
99974       671     4034     4.5  1064245493
99975       671     4306     5.0  1064245548
99976       671     4308     3.5  1065111985
99977       671     4880     4.0  1065111973
99978       671     4886     5.0  1064245488
99979       671     4896     5.0  1065111996
99980       671     4963     4.5  1065111855
99981       671     4973     4.5  1064245471
99982       671     4993     5.0  1064245483
99983       671     4995     4.0  1064891537
99984       671     5010     2.0  1066793004
99985       671     5218     2.0  1065111990
99986       671     5299     3.0  1065112004
99987       671     5349     4.0  1065111863
99988       671     5377     4.0  1064245557
99989       671     5445     4.5  1064891627
99990       671     5464     3.0  1064891549
99991       671     5669     4.0  1063502711
99992       671     5816     4.0  1065111963
99993       671     5902     3.5  1064245507
99994       671     5952     5.0  1063502716
99995       671     5989     4.0  1064890625
99996       671     5991     4.5  1064245387
99997       671     5995     4.0  1066793014
99998       671     6212     2.5  1065149436
99999       671     6268     2.5  1065579370
100000      671     6269     4.0  1065149201
100001      671     6365     4.0  1070940363
100002      671     6385     2.5  1070979663
100003      671     6565     3.5  1074784724

[100004 rows x 4 columns]

In [25]:
i = df_ratings['item_id']
np_movie_unique = np.unique(np.array(i), axis=0)

list_movie = []
for i, movie_id in enumerate(np_movie_unique):
    list_movie.append((i+1, movie_id))
    
df_mapping = pd.DataFrame(list_movie, columns=['movie_unique_id', 'item_id'])
df_mapping

movie_unique_id  item_id
0                   1        1
1                   2        2
2                   3        3
3                   4        4
4                   5        5
5                   6        6
6                   7        7
7                   8        8
8                   9        9
9                  10       10
10                 11       11
11                 12       12
12                 13       13
13                 14       14
14                 15       15
15                 16       16
16                 17       17
17                 18       18
18                 19       19
19                 20       20
20                 21       21
21                 22       22
22                 23       23
23                 24       24
24                 25       25
25                 26       26
26                 27       27
27                 28       28
28                 29       29
29                 30       30
...               ...      ...
9036             9037   158238
9037             9038   158314
9038             9039   158528
9039             9040   158956
9040             9041   159093
9041             9042   159462
9042             9043   159690
9043             9044   159755
9044             9045   159858
9045             9046   159972
9046             9047   160080
9047             9048   160271
9048             9049   160438
9049             9050   160440
9050             9051   160563
9051             9052   160565
9052             9053   160567
9053             9054   160590
9054             9055   160656
9055             9056   160718
9056             9057   161084
9057             9058   161155
9058             9059   161594
9059             9060   161830
9060             9061   161918
9061             9062   161944
9062             9063   162376
9063             9064   162542
9064             9065   162672
9065             9066   163949

[9066 rows x 2 columns]

In [26]:
df_join = pd.merge(df_ratings, df_mapping, on=['item_id'])

del df_join['item_id']
columns_new = ['user_id', 'movie_unique_id', 'rating', 'timestamp']
df_join = df_join[columns_new]
df = df_join.rename(columns={'movie_unique_id': 'item_id'})

df

user_id  item_id  rating   timestamp
0             1       31     2.5  1260759144
1             7       31     3.0   851868750
2            31       31     4.0  1273541953
3            32       31     4.0   834828440
4            36       31     3.0   847057202
5            39       31     3.0   832525157
6            73       31     3.5  1255591860
7            88       31     3.0  1239755559
8            96       31     2.5  1223256331
9           110       31     4.0   840100695
10          111       31     3.5  1097429230
11          150       31     2.5  1130905954
12          161       31     3.0   837629820
13          165       31     3.5  1111981801
14          186       31     3.0  1276205768
15          242       31     4.0   956686556
16          254       31     3.0   845157538
17          288       31     3.0   845862722
18          310       31     1.5  1414188058
19          311       31     3.0   898007906
20          325       31     4.5  1356316520
21          338       31     3.0   841656923
22          341       31     4.5  1240055463
23          386       31     2.0  1047028867
24          394       31     2.0  1297524887
25          452       31     2.5  1077114665
26          457       31     1.5  1471385561
27          461       31     2.0  1091959143
28          468       31     1.5  1296193889
29          485       31     4.0  1337748425
...         ...      ...     ...         ...
99974       652     8945     5.0  1440273477
99975       652     8955     5.0  1442690431
99976       652     8957     5.0  1442691006
99977       652     8982     3.0  1451790739
99978       652     8983     2.5  1449533204
99979       652     8984     3.0  1449533506
99980       652     8990     5.0  1447808622
99981       653     1664     3.0   948161066
99982       654     5424     4.5  1145391310
99983       654     5852     4.0  1145391359
99984       654     6303     3.0  1145391400
99985       655     8972     3.0  1470075909
99986       655     9038     4.5  1470071021
99987       656     1345     5.0   986242496
99988       656     3076     5.0   986242691
99989       656     3122     5.0   986244044
99990       659      147     4.0   836137550
99991       659      505     3.0   834694187
99992       663     8769     4.0  1438413063
99993       663     8858     3.0  1438412792
99994       663     8861     3.5  1438412783
99995       663     8863     3.0  1438412778
99996       663     8899     3.0  1438412878
99997       663     8902     3.5  1438412914
99998       664     6987     3.0  1393891653
99999       664     7104     2.5  1343761859
100000      664     7360     3.5  1344435977
100001      665      116     3.0   995232528
100002      665     3713     1.0  1010197684
100003      668     4630     1.0   993613478

[100004 rows x 4 columns]

In [27]:
import scipy
from scipy import sparse

r = df['rating'].astype(float)
u = df['user_id'].astype(int)
i = df['item_id'].astype(int)

mat_spar_rating = sparse.csr_matrix(
        (r, (u, i)),
        dtype=np.float
)

mat_spar_rating.shape

(672, 9067)

## ● 모델의 성능평가를 위해 위에서 만든 기본 데이터셋을 훈련/테스트 셋으로 나눈다.
###  실제 train / test 데이터를 반환하는 부분인 1)과 교차검증 함수를 정의한 2)로 나뉜다.
### 1) create train / test split (randomly)
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;무작위로 데이터를 train set과 test set으로 나눈다. ratio=0.25면 train:test (0.75:0.25)
### 2) create function for checking cross validation
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cross validation check 를 위한 함수 정의 
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;k_fold 이 10이면 train:test (0.9:0.1) 비율로 테스트를 10번 수행하고 평균값을 냄


### 1) randomly create train / test split

In [28]:
# split data to train(0.75) / test(0.25) by movies ranked for all users
# and make it sparse matrix

def convert_df_mat(df):
    r = df['rating'].astype(float)
    u = df['user_id'].astype(int)
    i = df['item_id'].astype(int)
    
    mat_spar = sparse.csr_matrix(
            (r, (u, i)),
            dtype=np.float
    )
    
    return mat_spar


def make_df_from_mat(mat_spar, num_movie):
    df_spar = pd.DataFrame(mat_spar.toarray())
    df_spar = df_spar.drop(df_spar.index[0], axis=0)
    del df_spar[0]
    
    if df_spar.shape[1] < num_movie:
        num_row = df_spar.shape[0]
        num_col = df_spar.shape[1]
        num_diff = num_movie - num_col
        
        np_add = np.zeros([num_row, num_diff])
        df_add = pd.DataFrame(np_add, index=np.arange(num_row) + 1, 
                              columns=(num_col + np.arange(num_diff) + 1))
        
        df_spar = pd.concat([df_spar, df_add], axis=1)
        
    return df_spar


def split_mat_random(mat_spar, ratio_test=0.25):
    n_1 = mat_spar.nonzero()[0]
    n_2 = mat_spar.nonzero()[1]
    np_sparse = np.dstack((n_1, n_2))[0, :, :]
    
    df_sparse_index = pd.DataFrame(np_sparse, columns=['user', 'movie'])
    df_group = df_sparse_index.groupby('user')['movie'].apply(np.array).reset_index(name='movies')
    
    list_movie_2d = [np_movie for np_movie in df_group['movies']]
    
    list_test = list(map(lambda x: np.random.choice(x, size=int(len(x) / int(1/ratio_test))), list_movie_2d))
    list_train = []
    for i, movies_all in enumerate(list_movie_2d):
        movies_test = list_test[i]
        movies_train = list(set(movies_all) - set(movies_test))
        
        list_train.append(movies_train)
        
    # np_test = list(map(lambda x: np.array_split(x, 4)[0], np_movie_2d))
    # np_train = list(map(lambda x: np.concatenate(np.array_split(x, 4)[1:]), np_movie_2d))
    
    lens_test = [len(movies) for movies in list_test]
    df_test = pd.DataFrame({'user_id': np.repeat(df_group['user'], lens_test),
                            'item_id': np.concatenate(list_test)})
    
    lens_train = [len(movies) for movies in list_train]
    df_train = pd.DataFrame({'user_id': np.repeat(df_group['user'], lens_train),
                             'item_id': np.concatenate(list_train)})
    
    df_train = pd.merge(df_train, df, on=['user_id', 'item_id'])
    df_test = pd.merge(df_test, df, on=['user_id', 'item_id'])
    
    mat_spar_train = convert_df_mat(df_train)
    mat_spar_test = convert_df_mat(df_test)
    
    return mat_spar_train, mat_spar_test
    

In [29]:
mat_spar_train, mat_spar_test = split_mat_random(mat_spar_rating)
df_spar_train = make_df_from_mat(mat_spar_train, mat_spar_rating.shape[1] - 1)
df_spar_test = make_df_from_mat(mat_spar_train, mat_spar_rating.shape[1] - 1)

In [30]:
print('shape of train: {}'.format(df_spar_train.shape))
print('shape of test: {}'.format(df_spar_test.shape))

shape of train: (671, 9066)
shape of test: (671, 9066)


### 2) creating function for checking cross validation

In [31]:
def convert_df_mat(df):
    r = df['rating'].astype(float)
    u = df['user_id'].astype(int)
    i = df['item_id'].astype(int)
    
    mat_spar = sparse.csr_matrix(
            (r, (u, i)),
            dtype=np.float
    )
    
    return mat_spar

def make_df_from_mat(mat_spar, num_movie):
    df_spar = pd.DataFrame(mat_spar.toarray())
    df_spar = df_spar.drop(df_spar.index[0], axis=0)
    del df_spar[0]
    
    if df_spar.shape[1] < num_movie:
        num_row = df_spar.shape[0]
        num_col = df_spar.shape[1]
        num_diff = num_movie - num_col
        
        np_add = np.zeros([num_row, num_diff])
        df_add = pd.DataFrame(np_add, index=np.arange(num_row) + 1, 
                              columns=(num_col + np.arange(num_diff) + 1))
        
        df_spar = pd.concat([df_spar, df_add], axis=1)
        
    return df_spar

def get_dataset(df, k_fold, offset):
    mat_spar = convert_df_mat(df)
    n_1 = mat_spar.nonzero()[0]
    n_2 = mat_spar.nonzero()[1]
    np_sparse = np.dstack((n_1, n_2))[0, :, :]
    
    df_sparse_index = pd.DataFrame(np_sparse, columns=['user', 'movie'])
    df_group = df_sparse_index.groupby('user')['movie'].apply(np.array).reset_index(name='movies')
    
    list_movie_2d = [np_movie for np_movie in df_group['movies']]
    
    list_test = list(map(lambda x: np.array_split(x, k_fold)[offset], list_movie_2d))
    list_train = []
    for i, movies_all in enumerate(list_movie_2d):
        movies_test = list_test[i]
        movies_train = list(set(movies_all) - set(movies_test))
        
        list_train.append(movies_train)
    
    lens_train = [len(movies) for movies in list_train]
    df_train_without_rating = pd.DataFrame({'user_id': np.repeat(df_group['user'], lens_train),
                             'item_id': np.concatenate(list_train)})

    lens_test = [len(movies) for movies in list_test]
    df_test_without_rating = pd.DataFrame({'user_id': np.repeat(df_group['user'], lens_test),
                            'item_id': np.concatenate(list_test)})
    
    df_train = pd.merge(df_train_without_rating, df, on=['user_id', 'item_id'])
    df_test = pd.merge(df_test_without_rating, df, on=['user_id', 'item_id'])
    
    mat_spar_train = convert_df_mat(df_train)
    mat_spar_test = convert_df_mat(df_test)

    num_items = mat_spar.shape[1]
    df_spar_train = make_df_from_mat(mat_spar_train, num_items - 1)
    df_spar_test = make_df_from_mat(mat_spar_test, num_items - 1)

    return df_spar_train, df_spar_test

def get_rmse(mat_spar_test, mat_spar_predict):
    def rmse(actual, prediction):
        tu_new = actual.nonzero()[0] - 1, actual.nonzero()[1] - 1
        pred = prediction[tu_new]
        actu = actual[tu_new]
        
        return sqrt(mean_squared_error(actu, pred))

    rmse = rmse(mat_spar_test, mat_spar_predict)
    print('rmse for item-based is {}'.format(rmse))

    return rmse


In [32]:
def cross_validation(df, model, k_fold=5, cores=4):
    list_validations = []
    for offset in range(k_fold):
        df_train, df_validation = get_dataset(df, k_fold, offset)
        np_result = model(df_train, cores)

        mat_spar_test = sparse.csr_matrix(df_validation)
        mat_spar_predict = sparse.csr_matrix(np_result)

        list_validations.append(get_rmse(mat_spar_test, mat_spar_predict))
        
    avg_rmse = np.mean(list_validations)
    return list_validations, avg_rmse


## =====================================================================
# 2. User-based CF (similarity algorithm = cosine)

## ● 모델 정의

### KNN

In [33]:
from sklearn.neighbors import NearestNeighbors

def findksimilarusers(user_id, df_spar, metric='cosine', k=5):
    similarities = []
    indices=[]
    model_knn = NearestNeighbors(metric=metric, algorithm='brute') 
    model_knn.fit(df_spar)

    distances, indices = model_knn.kneighbors(df_spar.iloc[user_id - 1, :].values.reshape(1, -1), n_neighbors = k)
    similarities = 1 - distances.flatten()

    return similarities, indices

### 특정 유저의 특정 영화에 대한 평점 예측 ( user:movie -> 1:1 )

In [34]:
def predict_userbased_user_item(user_id, item_id, df_spar, metric='cosine', k=5):
    prediction = 0
    similarities, indices = findksimilarusers(user_id, df_spar, metric, k) # similar users based on cosine similarity
    mean_rating = df_spar.iloc[user_id - 1, :].mean() # to adjust for zero based indexing
    sum_wt = np.sum(similarities) - 1
    product = 1
    wtd_sum = 0
    
    indices_flat = indices.flatten()
    for i, indice in enumerate(indices_flat):
        if indice + 1 == user_id:
            continue
        else:
            ratings_diff = df_spar.iloc[indice, item_id - 1] - np.mean(df_spar.iloc[indice, :])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    if sum_wt == 0:
        prediction = 0
    else:
        prediction = int(round(mean_rating + (wtd_sum / sum_wt)))
    
    return prediction

### 특정 유저의 모든 영화에 대한 평점 예측 ( user:movie -> 1:all )

In [35]:
def predict_userbased_user_allitems(user_id, df_spar, metric='cosine', k=5):
    similarities, indices = findksimilarusers(user_id, df_spar, metric, k)
    mean_rating = df_spar.iloc[user_id - 1, :].mean()
    
    list_pred_user = []
    indices_flat = indices.flatten()
    for item_id in range(1, df_spar.shape[1] + 1):
        sum_wt = np.sum(similarities) - 1
        product = 1
        wtd_sum = 0
        prediction = 0
        for i, indice in enumerate(indices_flat):
            if indice + 1 == user_id:
                continue
            else:
                rating_diff = df_spar.iloc[indice, item_id - 1] - np.mean(df_spar.iloc[indice, :])
                product = rating_diff * (similarities[i])
                wtd_sum = wtd_sum + product

        if sum_wt == 0:
            prediction = 0
        else:
            prediction = int(round(mean_rating + (wtd_sum / sum_wt)))
            
        list_pred_user.append(prediction)
    
    return list_pred_user

### Model 생성 (recommend_userbased)

In [42]:
def work(df_spar, id_user):
        return id_user, predict_userbased_user_allitems(id_user, df_spar)

def recommend_userbased(df_spar, cores=4):
    from multiprocessing import Pool
    from functools import partial
    import time
 
    time_start = time.time()
    
    p = Pool(processes=cores)
    iterable = list(range(1, df_spar.shape[0] + 1))
    func = partial(work, df_spar)
    result_pred = p.map(func, iterable)
    p.close()
    
    time_end = time.time()
    
    time_process = time_end - time_start
    print('processing time is: {} seconds'.format(time_process))
    
    list_pred_ratings_all = sorted(result_pred, key=lambda x: x[0])
    np_pred_ratings_all = np.array(list(map(lambda x: x[1], list_pred_ratings_all)))

    return np_pred_ratings_all

## ● 테스트 (using random data set created in step 1.)

### 특정 유저의 모든 영화에 대한 평점을 예측하는 데 걸리는 시간 (user : movie -> 1 : all) &nbsp;&nbsp;(영화수: 9066)

In [37]:
import time

time_start = time.time()
pred = predict_userbased_user_allitems(43, df_spar_train)
time_end = time.time()

print('한 유저의 모든 영화에 대한 평점을 예측하는 데 걸리는 시간: {} seconds'.format(time_end - time_start))
print('영화 수: {}'.format(len(pred)))

한 유저의 모든 영화에 대한 평점을 예측하는 데 걸리는 시간: 8.041028261184692 seconds
영화 수: 9066


### 모델을 돌려서 예측 값 매트릭스를 도출
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;싱글코어로 671 유저에 대한 평점을 예측하는 데 시간이 오래 걸림
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;싱글코어와 멀티코어 두가지 방법을 사용해서 시간을 측정하고 rmse 값을 비교 (동일한 결과 증명)

### Single-core

In [38]:
import time

time_start = time.time()

list_result = []
for i in range(1, df_spar_train.shape[0] + 1):
    list_pred_user = predict_userbased_user_allitems(i, df_spar_train)
    list_result.append(list_pred_user)

time_end = time.time()
time_process = time_end - time_start

print('process time for single core is: {} seconds'.format(time_process))
    
np_singlecore_result = np.array(list_result)
    

process time for single core is: 4336.494698047638 seconds


### Single-core RMSE

In [39]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0] - 1, actual.nonzero()[1] - 1
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_user = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_singlecore_result))
print('rmse for user-based is {}'.format(rmse_user))


rmse for user-based is 0.5851852641824478


### Multi-core

In [43]:
from multiprocessing import Pool
from functools import partial
import time

np_multicore_result = recommend_userbased(df_spar_train, cores=12)


processing time is: 1354.1436290740967 seconds


### Multi-core RMSE

In [130]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0], actual.nonzero()[1]
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_user = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_multicore_result))
print('rmse for user-based is {}'.format(rmse_user))

rmse for user-based is 2.4195351583498956


## ● Cross validation

In [45]:
# cross_validation(df, model, k_fold=5, cores=4)

list_validations, avg_rmse = cross_validation(df, recommend_userbased, k_fold=5, cores=12)
print('1. list of validations: {}'.format(list_validations))
print()
print('2. average of rmse: {}'.format(avg_rmse))

processing time is: 1430.518856048584 seconds
rmse for item-based is 0.8700279067086393
processing time is: 1462.0760560035706 seconds
rmse for item-based is 0.7050301048042926
processing time is: 1359.1902298927307 seconds
rmse for item-based is 0.5678812610317454
processing time is: 1349.71630525589 seconds
rmse for item-based is 0.45828062822050847
processing time is: 1334.771852016449 seconds
rmse for item-based is 0.46762767690326396
1. list of validations: [0.8700279067086393, 0.7050301048042926, 0.5678812610317454, 0.45828062822050847, 0.46762767690326396]

2. average of rmse: 0.6137695155336899


## ● 실제 추천
### 특정 유저에게 영화를 추천 (유저 기반)
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) 3점 이상의 예측 평점을 가진 영화 중 보지 않은 영화를 선별
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) 1)의 결과 중 예측 평점이 높은 순서, movie_id 가 빠른 순서대로 top 5의 영화를 추천

### 함수 정의

In [85]:
def get_list_seen_movie_id(df, id_user):
    mat_spar = convert_df_mat(df)
    sparse_nonzero = mat_spar.nonzero()
    list_id_movie_seen = []
    for i, id_user_exist_rating in enumerate(sparse_nonzero[0]):
        if id_user_exist_rating == id_user:
            list_id_movie_seen.append(sparse_nonzero[1][i])    

    return list_id_movie_seen

def get_df_movie_from_id(list_id_movie):
    np_insert = np.array(list_id_movie).reshape(-1, 1)
    df_id_movie = pd.DataFrame(np_insert, columns=['movie_unique_id'])
    df_joined = pd.merge(df_id_movie, df_mapping, on=['movie_unique_id'])
    df_movie = pd.merge(df_joined, df_movies, on=['item_id']).drop(columns=['movie_unique_id'])
    
    return df_movie

def recommend_item(df, id_user):
    mat_spar = convert_df_mat(df)
    df_spar = make_df_from_mat(mat_spar, 9066)
    list_pred_rank = predict_userbased_user_allitems(id_user, df_spar) 
    
    list_recommendation = []
    for i, rank in enumerate(list_pred_rank):
        if rank >= 3:
            if i + 1 not in get_list_seen_movie_id(df, id_user):
                list_recommendation.append(i+1)
    
    return list_recommendation

def get_seen_movie(df, id_user):
    return get_df_movie_from_id(get_list_seen_movie_id(df, id_user))

def get_recomm_movie(df, id_user):
    import time
    
    time_start = time.time()
    list_recomm = recommend_item(df, id_user)[:5]
    time_end = time.time()
    print('process time: {}'.format(time_end - time_start))

    return get_df_movie_from_id(list_recomm)

### 해당 유저가 본 영화목록을 보여준다

In [86]:
get_seen_movie(df, 3)

item_id                                              title  \
0        60                 Indian in the Cupboard, The (1995)   
1       110                                  Braveheart (1995)   
2       247                          Heavenly Creatures (1994)   
3       267                                 Major Payne (1995)   
4       296                                Pulp Fiction (1994)   
5       318                   Shawshank Redemption, The (1994)   
6       355                            Flintstones, The (1994)   
7       356                                Forrest Gump (1994)   
8       377                                       Speed (1994)   
9       527                            Schindler's List (1993)   
10      588                                     Aladdin (1992)   
11      592                                      Batman (1989)   
12      593                   Silence of the Lambs, The (1991)   
13      595                        Beauty and the Beast (1991)   
14      736                                     Twister (1996)   
15      778                               Trainspotting (1996)   
16      866                                       Bound (1996)   
17     1197                         Princess Bride, The (1987)   
18     1210  Star Wars: Episode VI - Return of the Jedi (1983)   
19     1235                            Harold and Maude (1971)   
20     1271                        Fried Green Tomatoes (1991)   
21     1378                                  Young Guns (1988)   
22     1580                   Men in Black (a.k.a. MIB) (1997)   
23     1721                                     Titanic (1997)   
24     1884              Fear and Loathing in Las Vegas (1998)   
25     2028                         Saving Private Ryan (1998)   
26     2318                                   Happiness (1998)   
27     2513                                Pet Sematary (1989)   
28     2694                                   Big Daddy (1999)   
29     2702                               Summer of Sam (1999)   
30     2716         Ghostbusters (a.k.a. Ghost Busters) (1984)   
31     2762                            Sixth Sense, The (1999)   
32     2841                              Stir of Echoes (1999)   
33     2858                             American Beauty (1999)   
34     2959                                  Fight Club (1999)   
35     3243                                  Encino Man (1992)   
36     3510                                   Frequency (2000)   
37     3949                         Requiem for a Dream (2000)   
38     5349                                  Spider-Man (2002)   
39     5669                       Bowling for Columbine (2002)   
40     6377                                Finding Nemo (2003)   
41     7153  Lord of the Rings: The Return of the King, The...   
42     7361       Eternal Sunshine of the Spotless Mind (2004)   
43     8622                             Fahrenheit 9/11 (2004)   
44     8636                                Spider-Man 2 (2004)   
45    27369                      Daria: Is It Fall Yet? (2000)   
46    44191                              V for Vendetta (2006)   
47    48783                        Flags of Our Fathers (2006)   
48    50068                       Letters from Iwo Jima (2006)   
49    58559                            Dark Knight, The (2008)   
50    84236  White Stripes Under Great White Northern Light...   

                                             genres  
0                        Adventure|Children|Fantasy  
1                                  Action|Drama|War  
2                                       Crime|Drama  
3                                            Comedy  
4                       Comedy|Crime|Drama|Thriller  
5                                       Crime|Drama  
6                           Children|Comedy|Fantasy  
7                          Comedy|Drama|Romance|War  
8                           Action|Romance|Thriller  
9                              

### 추천 시스템에 의해서 추천된 영화목록을 보여준다 (예측평점 상위 5개)

In [87]:
get_recomm_movie(df, 3)

process time: 10.486234188079834


item_id                 title  \
0        1      Toy Story (1995)   
1      150      Apollo 13 (1995)   
2      457  Fugitive, The (1993)   
3      480  Jurassic Park (1993)   
4     4306          Shrek (2001)   

                                              genres  
0        Adventure|Animation|Children|Comedy|Fantasy  
1                               Adventure|Drama|IMAX  
2                                           Thriller  
3                   Action|Adventure|Sci-Fi|Thriller  
4  Adventure|Animation|Children|Comedy|Fantasy|Ro...

## =====================================================================
# 3. Item-based CF

# 3-1. Item-based (cosine)

## ● 모델 정의

### KNN

In [49]:
from sklearn.neighbors import NearestNeighbors

def find_similar_items(item_id, df_spar, metric='cosine', k=5):
    df_T = df_spar.T
    model_knn = NearestNeighbors(metric=metric, algorithm='brute') 
    model_knn.fit(df_T)

    similarities, indices = model_knn.kneighbors(df_T.iloc[item_id - 1, :].values.reshape(1, -1), n_neighbors = k)
    similarities = 1 - similarities.flatten()

    return similarities, indices

### 특정 유저의 특정 영화에 대한 평점 예측 (user:movie -> 1:1)

In [50]:
def predict_itembased_user_item(user_id, item_id, df_spar, metric='cosine', k=5):
    prediction = 0
    similarities, indices = find_similar_items(item_id, df_spar, metric, k) 
    sum_wt = np.sum(similarities) - 1
    product = 1
    wtd_sum = 0
    
    indices_flat = indices.flatten()
    for i, indice in enumerate(indices_flat):
        if indice + 1 == item_id:
            continue
        else:
            product = df_spar.iloc[user_id - 1, indice] * (similarities[i])
            wtd_sum = wtd_sum + product

    if sum_wt == 0:
        prediction = 0
    else:
        prediction = int(round(wtd_sum / sum_wt))
            
    return prediction

### 특정 영화의 모든 유저에 대한 평점 예측 (user:movie -> all:1) 

In [51]:
def predict_itembased_item_allusers(item_id, df_spar, metric='cosine', k=5):
    similarities, indices = find_similar_items(item_id, df_spar, metric, k)
    
    list_pred_all_rating = []
    indices_flat = indices.flatten()
    for user_id in range(1, df_spar.shape[0] + 1):
        prediction = 0
        sum_wt = np.sum(similarities) - 1
        product = 1
        wtd_sum = 0
    
        indices_flat = indices.flatten()
        for i, indice in enumerate(indices_flat):
            if indice + 1 == item_id:
                continue
            else:
                product = df_spar.iloc[user_id - 1, indice] * (similarities[i])
                wtd_sum = wtd_sum + product

        if sum_wt == 0:
            prediction = 0
        else:
            prediction = int(round(wtd_sum / sum_wt))
        
        list_pred_all_rating.append(prediction)
    
    return list_pred_all_rating
    
    

### Model 생성 (recommend_itembased)

In [99]:
def work(df_spar, id_item):
        return id_item, predict_itembased_item_allusers(id_item, df_spar)

def recommend_itembased(df_spar, cores=4):
    from multiprocessing import Pool
    from functools import partial
    import time
 
    time_start = time.time()
    
    p = Pool(processes=cores)
    iterable = list(range(1, df_spar.shape[1] + 1))
    func = partial(work, df_spar)
    result_pred = p.map(func, iterable)
    p.close()
    
    time_end = time.time()
    
    time_process = time_end - time_start
    print('processing time is: {} seconds'.format(time_process))
    
    list_pred_ratings_all = sorted(result_pred, key=lambda x: x[0])
    np_pred_ratings_all = np.array(list(map(lambda x: x[1], list_pred_ratings_all)))

    return np_pred_ratings_all.T

## ● 테스트 (using random data set created in step 1.)

### 특정 영화의 모든 유저에 대한 평점 예측하는 데 걸리는 시간 (user : movie -> 1 : all) &nbsp;&nbsp;(유저수: 671)

In [53]:
import time

time_start = time.time()
predict_itembased_item_allusers(9063, df_spar_train)
time_end = time.time()

print('process time: {} seconds'.format(time_end - time_start))

process time: 0.14527487754821777 seconds


### Multi-core

In [55]:
from multiprocessing import Pool
from functools import partial
import time

np_multicore_result = recommend_itembased(df_spar_train, cores=4)


processing time is: 331.95966124534607 seconds


### Multi-core RMSE

In [131]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0], actual.nonzero()[1]
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_item = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_multicore_result))
print('rmse for item-based is {}'.format(rmse_item))

rmse for item-based is 2.4195351583498956


## ● Cross validation

#### k=5

In [57]:
list_validations, avg_rmse =  cross_validation(df, recommend_itembased, k_fold=5, cores=12)

processing time is: 279.9190180301666 seconds
rmse for item-based is 0.847609255168369
processing time is: 277.76744771003723 seconds
rmse for item-based is 0.6655370086749495
processing time is: 270.00314688682556 seconds
rmse for item-based is 0.5043305935573018
processing time is: 271.35660910606384 seconds
rmse for item-based is 0.4355942653565045
processing time is: 261.707701921463 seconds
rmse for item-based is 0.41940115885727797


In [58]:
print('1. list of validations: \n{}'.format(list_validations))
print()
print('2. average of rmse: \n{}'.format(avg_rmse))

1. list of validations: 
[0.847609255168369, 0.6655370086749495, 0.5043305935573018, 0.4355942653565045, 0.41940115885727797]

2. average of rmse: 
0.5744944563228807


## ● 실제 추천
### 특정 유저에게 영화를 추천 (아이템 기반)
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) 3점 이상의 예측 평점을 가진 영화 중 보지 않은 영화를 선별
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) 1)의 결과 중 예측 평점이 높은 순서, movie_id 가 빠른 순서대로 top 5의 영화를 추천

### 함수 정의

In [104]:
def get_list_seen_movie_id(df, id_user):
    mat_spar = convert_df_mat(df)
    sparse_nonzero = mat_spar.nonzero()
    list_id_movie_seen = []
    for i, id_user_exist_rating in enumerate(sparse_nonzero[0]):
        if id_user_exist_rating == id_user:
            list_id_movie_seen.append(sparse_nonzero[1][i])    

    return list_id_movie_seen

def get_df_movie_from_id(list_id_movie):
    np_insert = np.array(list_id_movie).reshape(-1, 1)
    df_id_movie = pd.DataFrame(np_insert, columns=['movie_unique_id'])
    df_joined = pd.merge(df_id_movie, df_mapping, on=['movie_unique_id'])
    df_movie = pd.merge(df_joined, df_movies, on=['item_id']).drop(columns=['movie_unique_id'])
    
    return df_movie

def recommend_item(df, id_user):
    mat_spar = convert_df_mat(df)
    df_spar = make_df_from_mat(mat_spar, 9066)
    np_multicore_result = recommend_itembased(df_spar, cores=12)
    
    list_recommendation = []
    for i, rank in enumerate(np_multicore_result[id_user - 1]):
        if rank >= 3:
            if i + 1 not in get_list_seen_movie_id(df, id_user):
                list_recommendation.append(i+1)
    
    return list_recommendation

def get_seen_movie(df, id_user):
    return get_df_movie_from_id(get_list_seen_movie_id(df, id_user))

def get_recomm_movie(df, id_user):
    import time
    
    time_start = time.time()
    list_recomm = recommend_item(df, id_user)[:5]
    time_end = time.time()
    print('process time: {}'.format(time_end - time_start))

    return get_df_movie_from_id(list_recomm)

### 해당 유저가 본 영화목록을 보여준다

In [105]:
get_seen_movie(df, 3)

item_id                                              title  \
0        60                 Indian in the Cupboard, The (1995)   
1       110                                  Braveheart (1995)   
2       247                          Heavenly Creatures (1994)   
3       267                                 Major Payne (1995)   
4       296                                Pulp Fiction (1994)   
5       318                   Shawshank Redemption, The (1994)   
6       355                            Flintstones, The (1994)   
7       356                                Forrest Gump (1994)   
8       377                                       Speed (1994)   
9       527                            Schindler's List (1993)   
10      588                                     Aladdin (1992)   
11      592                                      Batman (1989)   
12      593                   Silence of the Lambs, The (1991)   
13      595                        Beauty and the Beast (1991)   
14      736                                     Twister (1996)   
15      778                               Trainspotting (1996)   
16      866                                       Bound (1996)   
17     1197                         Princess Bride, The (1987)   
18     1210  Star Wars: Episode VI - Return of the Jedi (1983)   
19     1235                            Harold and Maude (1971)   
20     1271                        Fried Green Tomatoes (1991)   
21     1378                                  Young Guns (1988)   
22     1580                   Men in Black (a.k.a. MIB) (1997)   
23     1721                                     Titanic (1997)   
24     1884              Fear and Loathing in Las Vegas (1998)   
25     2028                         Saving Private Ryan (1998)   
26     2318                                   Happiness (1998)   
27     2513                                Pet Sematary (1989)   
28     2694                                   Big Daddy (1999)   
29     2702                               Summer of Sam (1999)   
30     2716         Ghostbusters (a.k.a. Ghost Busters) (1984)   
31     2762                            Sixth Sense, The (1999)   
32     2841                              Stir of Echoes (1999)   
33     2858                             American Beauty (1999)   
34     2959                                  Fight Club (1999)   
35     3243                                  Encino Man (1992)   
36     3510                                   Frequency (2000)   
37     3949                         Requiem for a Dream (2000)   
38     5349                                  Spider-Man (2002)   
39     5669                       Bowling for Columbine (2002)   
40     6377                                Finding Nemo (2003)   
41     7153  Lord of the Rings: The Return of the King, The...   
42     7361       Eternal Sunshine of the Spotless Mind (2004)   
43     8622                             Fahrenheit 9/11 (2004)   
44     8636                                Spider-Man 2 (2004)   
45    27369                      Daria: Is It Fall Yet? (2000)   
46    44191                              V for Vendetta (2006)   
47    48783                        Flags of Our Fathers (2006)   
48    50068                       Letters from Iwo Jima (2006)   
49    58559                            Dark Knight, The (2008)   
50    84236  White Stripes Under Great White Northern Light...   

                                             genres  
0                        Adventure|Children|Fantasy  
1                                  Action|Drama|War  
2                                       Crime|Drama  
3                                            Comedy  
4                       Comedy|Crime|Drama|Thriller  
5                                       Crime|Drama  
6                           Children|Comedy|Fantasy  
7                          Comedy|Drama|Romance|War  
8                           Action|Romance|Thriller  
9                              

### 추천 시스템에 의해서 추천된 영화목록을 보여준다 (예측평점 상위 5개)

In [106]:
get_recomm_movie(df, 3)

processing time is: 268.2513756752014 seconds
process time: 270.41912293434143


item_id                                              title  \
0       47                        Seven (a.k.a. Se7en) (1995)   
1       50                         Usual Suspects, The (1995)   
2      293  Léon: The Professional (a.k.a. The Professiona...   
3     2329                          American History X (1998)   

                        genres  
0             Mystery|Thriller  
1       Crime|Mystery|Thriller  
2  Action|Crime|Drama|Thriller  
3                  Crime|Drama

## =====================================================================

# 3-2. Item-based (adjusted cosine)

#### user_A = [0 0 0 5 6 0]
#### user_B = [0 2 3 4 0 0]
#### 두 유저에 대한 아이템 평가가 위와 같이 있을 때 adj cosine 을 적용하는 방법
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;case 1. 모든 평점 값에(0 포함) 평점평균값을 빼준 후 벡터간의 cosine similarity를 계산
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;case 2. 평점이 있는 값들만(0 제외) 평점평균값을 빼준 후 벡터간의 cosine similarity를 계산




### case 1과 2를 각각 돌려보고 rmse 값의 차이를 확인한다.

## ● 모델 정의

### KNN

In [59]:
def find_similar_items_adjcos(sim_matrix, id_item, k=5):
    np_top_k = sim_matrix[id_item-1].sort_values(ascending=False)[:k+1]
    similarities = np_top_k.values
    indices = np_top_k.index

    return similarities, indices



### 특정 유저의 특정 영화에 대한 평점 예측 (user:movie -> 1:1)

In [60]:
def predict_itembased_user_item_adjcos(sim_matrix, id_user, id_item, k=5):
    prediction = wtd_sum = 0
    product = 1
    similarities, indices = find_similar_items_adjcos(sim_matrix, id_item, k) 
    sum_wt = np.sum(similarities) - 1
    
    for i, indice in enumerate(indices):
        if indice + 1 == item_id:
            continue
        else:
            product = df_spar.iloc[id_user - 1, indice] * (similarities[i])
            wtd_sum = wtd_sum + product
            
    if sum_wt == 0:
        prediction = 0
    else:
        prediction = int(round(wtd_sum / sum_wt))

    return prediction

### 특정 영화의 모든 유저에 대한 평점 예측 (user:movie -> all:1) 

In [61]:
def predict_itembased_item_all_users_adjcos(df_spar, sim_matrix, id_item, k=5):
    similarities, indices = find_similar_items_adjcos(sim_matrix, id_item, k)
    
    list_pred_all_rating = []
    for id_user in range(1, df_spar.shape[0] + 1):
        prediction = 0
        sum_wt = np.sum(similarities) - 1
        product = 1
        wtd_sum = 0
        
        for i, indice in enumerate(indices):
            if indice + 1 == id_item:
                continue
            else:
                product = df_spar.iloc[id_user - 1, indice] * (similarities[i])
                wtd_sum = wtd_sum + product
        
        if sum_wt == 0:
            prediction = 0
        else:
            prediction = int(round(wtd_sum / sum_wt))
            
        list_pred_all_rating.append(prediction)
        
    return list_pred_all_rating


### adj cosine 유사도 매트릭스

#### case 1.

In [62]:
import time
from scipy.spatial.distance import pdist, squareform

def get_sim_matrix_sub_all(df_spar):
    M = np.array(df_spar)
    
    M_u = M.mean(axis=1)
    M_adj = M - M_u[:, None]
    
    time_start = time.time()
    sim_matrix_sub_all = 1 - squareform(pdist(M_adj.T, 'cosine'))
    time_end = time.time()
    
    print(time_end - time_start)
    sim_matrix_sub_all = pd.DataFrame(sim_matrix_sub_all)
    sim_matrix_sub_all
    
    return sim_matrix_sub_all

#### case 2.

In [63]:
import time
from scipy.spatial.distance import pdist, squareform

def get_sim_matrix_sub_exist(df_spar):
    M = np.array(df_spar)
    list_mean = []
    for rating_user in M:
        np_idx_nonzero = rating_user.nonzero()[0]
        mean = np.mean(rating_user[np_idx_nonzero])
        list_mean.append(mean)
    M_u = np.array(list_mean)
    
    M_adj = np.zeros([df_spar.shape[0], df_spar.shape[1]])

    for row, np_ratings in enumerate(M):
        np_idx_nonzero = np_ratings.nonzero()[0]
        M_adj[row][np_idx_nonzero] = np_ratings[np_idx_nonzero] - M_u[row]
    
    time_start = time.time()
    sim_matrix_sub_exist = 1 - squareform(pdist(M_adj.T, 'cosine'))
    time_end = time.time()
    print(time_end - time_start)
    
    mask = np.isnan(sim_matrix_sub_exist)
    sim_matrix_sub_exist[mask] = 0
    
    sim_matrix_sub_exist = pd.DataFrame(sim_matrix_sub_exist)
    
    return sim_matrix_sub_exist

### 모델 생성 (recommend_itembased_adjcosine)

#### case 1.

In [64]:
def work_all(df_spar, sim_matrix, id_item):
    return id_item, predict_itembased_item_all_users_adjcos(df_spar, sim_matrix, id_item)

def recommend_itembased_adjcosine_all(df_spar, cores=4):
    from multiprocessing import Pool
    from functools import partial
    import time
 
    time_start = time.time()

    sim_matrix = get_sim_matrix_sub_all(df_spar)
    p = Pool(processes=cores)
    iterable = list(range(1, df_spar.shape[1] + 1))
    func = partial(work_all, df_spar, sim_matrix)
    result_pred = p.map(func, iterable)
    p.close()
    
    time_end = time.time()
    
    time_process = time_end - time_start
    print('processing time is: {} seconds'.format(time_process))
    
    list_pred_ratings_all = sorted(result_pred, key=lambda x: x[0])
    np_pred_ratings_all = np.array(list(map(lambda x: x[1], list_pred_ratings_all)))

    return np_pred_ratings_all.T

#### case 2.

In [65]:
def work_exist(df_spar, sim_matrix, id_item):
    return id_item, predict_itembased_item_all_users_adjcos(df_spar, sim_matrix, id_item)

def recommend_itembased_adjcosine_exist(df_spar, cores=4):
    from multiprocessing import Pool
    from functools import partial
    import time
 
    time_start = time.time()

    sim_matrix = get_sim_matrix_sub_exist(df_spar)
    p = Pool(processes=cores)
    iterable = list(range(1, df_spar.shape[1] + 1))
    func = partial(work_exist, df_spar, sim_matrix)
    result_pred = p.map(func, iterable)
    p.close()
    
    time_end = time.time()
    
    time_process = time_end - time_start
    print('processing time is: {} seconds'.format(time_process))
    
    list_pred_ratings_exist = sorted(result_pred, key=lambda x: x[0])
    list_pred_ratings_exist = np.array(list(map(lambda x: x[1], list_pred_ratings_exist)))

    return list_pred_ratings_exist.T

## ● 테스트 (using random data set created in step 1.)

### 특정 영화의 모든 유저에 대한 평점 예측하는 데 걸리는 시간 (user : movie -> 1 : all) &nbsp;&nbsp;(유저수: 671)

#### case 1.

In [66]:
import time

time_start = time.time()
sim_matrix = get_sim_matrix_sub_all(df_spar_train)
predict_itembased_item_all_users_adjcos(df_spar_train, sim_matrix, 9063, 5)
time_end = time.time()

print('process time (subtract all): {} seconds'.format(time_end - time_start))

30.034446001052856
process time (subtract all): 30.118995904922485 seconds


#### case 2.

In [67]:
import time

time_start = time.time()
sim_matrix = get_sim_matrix_sub_exist(df_spar_train)
predict_itembased_item_all_users_adjcos(df_spar_train, sim_matrix, 9063, 5)
time_end = time.time()

print('process time (subtract exist): {} seconds'.format(time_end - time_start))

30.06711173057556
process time (subtract exist): 30.573616981506348 seconds


### Multi-core

#### case 1.

In [68]:
np_multicore_result_T_all = recommend_itembased_adjcosine_all(df_spar_train, cores=12)

29.984957218170166
processing time is: 221.62176704406738 seconds


#### case 2.

In [69]:
np_multicore_result_T_exist = recommend_itembased_adjcosine_exist(df_spar_train, cores=12)

30.09826922416687
processing time is: 215.94884490966797 seconds


### Multi-core RMSE

#### case 1.

In [132]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0], actual.nonzero()[1]
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_user = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_multicore_result_T_all))
print('rmse for item-based is {}'.format(rmse_user))

rmse for item-based is 2.3175792296557756


#### case 2.

In [71]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0] - 1, actual.nonzero()[1] - 1
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_user = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_multicore_result_T_exist))
print('rmse for item-based is {}'.format(rmse_user))

rmse for item-based is 0.5609812522739072


## ● Cross validation

#### case 1.

In [72]:
list_validations_all, avg_rmse_all =  cross_validation(df, recommend_itembased_adjcosine_all, k_fold=5, cores=12)

29.942795038223267
processing time is: 217.62825417518616 seconds
rmse for item-based is 0.8463269188407647
29.985151052474976
processing time is: 233.05463695526123 seconds
rmse for item-based is 0.6633676679308442
29.8596351146698
processing time is: 217.72955083847046 seconds
rmse for item-based is 0.5111734590666968
29.808252096176147
processing time is: 218.79511213302612 seconds
rmse for item-based is 0.43738114889734137
29.63261079788208
processing time is: 211.875155210495 seconds
rmse for item-based is 0.4183133730399872


In [73]:
print('1. list of validations: \n{}'.format(list_validations_all))
print()
print('2. average of rmse: \n{}'.format(avg_rmse_all))

1. list of validations: 
[0.8463269188407647, 0.6633676679308442, 0.5111734590666968, 0.43738114889734137, 0.4183133730399872]

2. average of rmse: 
0.5753125135551269


#### case 2.

In [74]:
list_validations_exist, avg_rmse_exist =  cross_validation(df, recommend_itembased_adjcosine_exist, k_fold=5, cores=12)

29.929625034332275
processing time is: 219.63015699386597 seconds
rmse for item-based is 0.7728787281559889
29.999388933181763
processing time is: 224.47297191619873 seconds
rmse for item-based is 0.5648685642303076
29.545958995819092
processing time is: 218.4144651889801 seconds
rmse for item-based is 0.3987024648176661
29.759747982025146
processing time is: 218.1958041191101 seconds
rmse for item-based is 0.3339847540400054
29.713716983795166
processing time is: 215.2415030002594 seconds
rmse for item-based is 0.336712315658973


In [75]:
print('1. list of validations: \n{}'.format(list_validations_exist))
print()
print('2. average of rmse: \n{}'.format(avg_rmse_exist))

1. list of validations: 
[0.7728787281559889, 0.5648685642303076, 0.3987024648176661, 0.3339847540400054, 0.336712315658973]

2. average of rmse: 
0.48142936538058817


## ● 실제 추천
### 특정 유저에게 영화를 추천 (adj cosine 아이템 기반)
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) 3점 이상의 예측 평점을 가진 영화 중 보지 않은 영화를 선별
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) 1)의 결과 중 예측 평점이 높은 순서, movie_id 가 빠른 순서대로 top 5의 영화를 추천

### 함수 정의

In [112]:
def get_list_seen_movie_id(df, id_user):
    mat_spar = convert_df_mat(df)
    sparse_nonzero = mat_spar.nonzero()
    list_id_movie_seen = []
    for i, id_user_exist_rating in enumerate(sparse_nonzero[0]):
        if id_user_exist_rating == id_user:
            list_id_movie_seen.append(sparse_nonzero[1][i])    

    return list_id_movie_seen

def get_df_movie_from_id(list_id_movie):
    np_insert = np.array(list_id_movie).reshape(-1, 1)
    df_id_movie = pd.DataFrame(np_insert, columns=['movie_unique_id'])
    df_joined = pd.merge(df_id_movie, df_mapping, on=['movie_unique_id'])
    df_movie = pd.merge(df_joined, df_movies, on=['item_id']).drop(columns=['movie_unique_id'])
    
    return df_movie

def recommend_item_all(df, id_user):
    mat_spar = convert_df_mat(df)
    df_spar = make_df_from_mat(mat_spar, 9066)
    np_multicore_result = recommend_itembased_adjcosine_all(df_spar, cores=12)
    
    list_recommendation = []
    for i, rank in enumerate(np_multicore_result[id_user - 1]):
        if rank >= 3:
            if i + 1 not in get_list_seen_movie_id(df, id_user):
                list_recommendation.append(i+1)
    
    return list_recommendation

def recommend_item_exist(df, id_user):
    mat_spar = convert_df_mat(df)
    df_spar = make_df_from_mat(mat_spar, 9066)
    np_multicore_result = recommend_itembased_adjcosine_exist(df_spar, cores=12)
    
    list_recommendation = []
    for i, rank in enumerate(np_multicore_result[id_user - 1]):
        if rank >= 3:
            if i + 1 not in get_list_seen_movie_id(df, id_user):
                list_recommendation.append(i+1)
    
    return list_recommendation

def get_seen_movie(df, id_user):
    return get_df_movie_from_id(get_list_seen_movie_id(df, id_user))

def get_recomm_movie_all(df, id_user):
    import time
    
    time_start = time.time()
    list_recomm = recommend_item_all(df, id_user)[:5]
    time_end = time.time()
    print('process time: {}'.format(time_end - time_start))

    return get_df_movie_from_id(list_recomm)

def get_recomm_movie_exist(df, id_user):
    import time
    
    time_start = time.time()
    list_recomm = recommend_item_exist(df, id_user)[:5]
    time_end = time.time()
    print('process time: {}'.format(time_end - time_start))

    return get_df_movie_from_id(list_recomm)

### 해당 유저가 본 영화목록을 보여준다

In [113]:
get_seen_movie(df, 3)

item_id                                              title  \
0        60                 Indian in the Cupboard, The (1995)   
1       110                                  Braveheart (1995)   
2       247                          Heavenly Creatures (1994)   
3       267                                 Major Payne (1995)   
4       296                                Pulp Fiction (1994)   
5       318                   Shawshank Redemption, The (1994)   
6       355                            Flintstones, The (1994)   
7       356                                Forrest Gump (1994)   
8       377                                       Speed (1994)   
9       527                            Schindler's List (1993)   
10      588                                     Aladdin (1992)   
11      592                                      Batman (1989)   
12      593                   Silence of the Lambs, The (1991)   
13      595                        Beauty and the Beast (1991)   
14      736                                     Twister (1996)   
15      778                               Trainspotting (1996)   
16      866                                       Bound (1996)   
17     1197                         Princess Bride, The (1987)   
18     1210  Star Wars: Episode VI - Return of the Jedi (1983)   
19     1235                            Harold and Maude (1971)   
20     1271                        Fried Green Tomatoes (1991)   
21     1378                                  Young Guns (1988)   
22     1580                   Men in Black (a.k.a. MIB) (1997)   
23     1721                                     Titanic (1997)   
24     1884              Fear and Loathing in Las Vegas (1998)   
25     2028                         Saving Private Ryan (1998)   
26     2318                                   Happiness (1998)   
27     2513                                Pet Sematary (1989)   
28     2694                                   Big Daddy (1999)   
29     2702                               Summer of Sam (1999)   
30     2716         Ghostbusters (a.k.a. Ghost Busters) (1984)   
31     2762                            Sixth Sense, The (1999)   
32     2841                              Stir of Echoes (1999)   
33     2858                             American Beauty (1999)   
34     2959                                  Fight Club (1999)   
35     3243                                  Encino Man (1992)   
36     3510                                   Frequency (2000)   
37     3949                         Requiem for a Dream (2000)   
38     5349                                  Spider-Man (2002)   
39     5669                       Bowling for Columbine (2002)   
40     6377                                Finding Nemo (2003)   
41     7153  Lord of the Rings: The Return of the King, The...   
42     7361       Eternal Sunshine of the Spotless Mind (2004)   
43     8622                             Fahrenheit 9/11 (2004)   
44     8636                                Spider-Man 2 (2004)   
45    27369                      Daria: Is It Fall Yet? (2000)   
46    44191                              V for Vendetta (2006)   
47    48783                        Flags of Our Fathers (2006)   
48    50068                       Letters from Iwo Jima (2006)   
49    58559                            Dark Knight, The (2008)   
50    84236  White Stripes Under Great White Northern Light...   

                                             genres  
0                        Adventure|Children|Fantasy  
1                                  Action|Drama|War  
2                                       Crime|Drama  
3                                            Comedy  
4                       Comedy|Crime|Drama|Thriller  
5                                       Crime|Drama  
6                           Children|Comedy|Fantasy  
7                          Comedy|Drama|Romance|War  
8                           Action|Romance|Thriller  
9                              

### 추천 시스템에 의해서 추천된 영화목록을 보여준다 (예측평점 상위 5개)

In [114]:
get_recomm_movie_all(df, 3)

30.023032188415527
processing time is: 217.78400373458862 seconds
process time: 219.9251048564911


item_id                                              title  \
0       47                        Seven (a.k.a. Se7en) (1995)   
1       50                         Usual Suspects, The (1995)   
2      293  Léon: The Professional (a.k.a. The Professiona...   
3     2329                          American History X (1998)   

                        genres  
0             Mystery|Thriller  
1       Crime|Mystery|Thriller  
2  Action|Crime|Drama|Thriller  
3                  Crime|Drama

In [115]:
get_recomm_movie_exist(df, 3)

30.033227682113647
processing time is: 220.1394009590149 seconds
process time: 221.13793992996216


Empty DataFrame
Columns: [item_id, title, genres]
Index: []

# 4. SVD

## ● 모델 정의

In [147]:
def model_svd(df_spar, k_input, val_adj):
    np_spar = np.array(df_spar)
    mean_user_ratings = np.mean(np_spar, axis = 1)
    np_train_demeaned = np_spar - mean_user_ratings.reshape(-1, 1)
    
    from scipy.sparse.linalg import svds
    u, sigma, vt = svds(np_train_demeaned, k = k_input)
    
    sigma = np.diag(sigma)
    
    np_pred_ratings_all_users = np.dot(np.dot(u, sigma), vt) + mean_user_ratings.reshape(-1, 1) + val_adj
    
    return np_pred_ratings_all_users

## ● 테스트

In [155]:
np_preds = model_svd(df_spar_train, 1, 3.2)
np_preds.shape
        

(671, 9066)

## ● RMSE

In [156]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import sparse

def rmse(actual, prediction):
    tu_new = actual.nonzero()[0], actual.nonzero()[1]
    pred = prediction[tu_new]
    actu = actual[tu_new]
    
    return sqrt(mean_squared_error(actu, pred))

rmse_user = rmse(sparse.csr_matrix(df_spar_test), sparse.csr_matrix(np_preds))
print('rmse for item-based is {}'.format(rmse_user))

rmse for item-based is 1.1928929520976053


## ● Cross validation

In [159]:
def cv_svd(df, k_fold=5, k_svd=50, value_adj=3):
    list_validations = []
    for offset in range(k_fold):
        df_train, df_validation = get_dataset(df, k_fold, offset)
        np_result = model_svd(df_train, k_svd, value_adj)
    
        mat_spar_test = sparse.csr_matrix(df_validation)
        mat_spar_predict = sparse.csr_matrix(np_result)
    
        list_validations.append(rmse(mat_spar_test, mat_spar_predict))
        
    avg_rmse = np.mean(list_validations)
    
    return list_validations, avg_rmse

list_validations, avg_rmse = cv_svd(df, 5, 50, 3.2)

print('1. list of validations: \n{}'.format(list_validations))
print()
print('2. average of rmse: \n{}'.format(avg_rmse))

1. list of validations: 
[1.1348973455581102, 1.1106192569260647, 1.0938477221618759, 1.0865253776808304, 1.0732549123961361]

2. average of rmse: 
1.0998289229446034


### k 값 증가시켜 가며 테스트 1 ~ 100

In [162]:
np_val_adj = np.array(list(range(10, 41))) / 10
k_best = float('inf')
for k in range(1, 101):
    for val_adj in np_val_adj:
        avg_rmse = cv_svd(df, k_fold=5, k_svd=k, value_adj=val_adj)[1]
        if avg_rmse <= k_best:
            k_best = avg_rmse
            i_best = k, val_adj
            
        
print('best k is: {}'.format(i_best))

best k is: (100, 3.4)


## ● 실제 추천
### 특정 유저에게 영화를 추천 (svd)
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) 3점 이상의 예측 평점을 가진 영화 중 보지 않은 영화를 선별
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) 1)의 결과 중 예측 평점이 높은 순서, movie_id 가 빠른 순서대로 top 5의 영화를 추천

### 함수 정의

In [122]:
def get_list_seen_movie_id(df, id_user):
    mat_spar = convert_df_mat(df)
    sparse_nonzero = mat_spar.nonzero()
    list_id_movie_seen = []
    for i, id_user_exist_rating in enumerate(sparse_nonzero[0]):
        if id_user_exist_rating == id_user:
            list_id_movie_seen.append(sparse_nonzero[1][i])    

    return list_id_movie_seen

def get_df_movie_from_id(list_id_movie):
    np_insert = np.array(list_id_movie).reshape(-1, 1)
    df_id_movie = pd.DataFrame(np_insert, columns=['movie_unique_id'])
    df_joined = pd.merge(df_id_movie, df_mapping, on=['movie_unique_id'])
    df_movie = pd.merge(df_joined, df_movies, on=['item_id']).drop(columns=['movie_unique_id'])
    
    return df_movie

def recommend_item(df, id_user):
    mat_spar = convert_df_mat(df)
    df_spar = make_df_from_mat(mat_spar, 9066)
    np_multicore_result = model_svd(df_spar, 1)
    
    list_recommendation = []
    for i, rank in enumerate(np_multicore_result[id_user - 1]):
        if rank >= 3:
            if i + 1 not in get_list_seen_movie_id(df, id_user):
                list_recommendation.append(i+1)
    
    return list_recommendation

def get_seen_movie(df, id_user):
    return get_df_movie_from_id(get_list_seen_movie_id(df, id_user))

def get_recomm_movie(df, id_user):
    import time
    
    time_start = time.time()
    list_recomm = recommend_item(df, id_user)[:5]
    time_end = time.time()
    print('process time: {}'.format(time_end - time_start))

    return get_df_movie_from_id(list_recomm)

### 해당 유저가 본 영화목록을 보여준다

In [123]:
get_seen_movie(df, 3)

item_id                                              title  \
0        60                 Indian in the Cupboard, The (1995)   
1       110                                  Braveheart (1995)   
2       247                          Heavenly Creatures (1994)   
3       267                                 Major Payne (1995)   
4       296                                Pulp Fiction (1994)   
5       318                   Shawshank Redemption, The (1994)   
6       355                            Flintstones, The (1994)   
7       356                                Forrest Gump (1994)   
8       377                                       Speed (1994)   
9       527                            Schindler's List (1993)   
10      588                                     Aladdin (1992)   
11      592                                      Batman (1989)   
12      593                   Silence of the Lambs, The (1991)   
13      595                        Beauty and the Beast (1991)   
14      736                                     Twister (1996)   
15      778                               Trainspotting (1996)   
16      866                                       Bound (1996)   
17     1197                         Princess Bride, The (1987)   
18     1210  Star Wars: Episode VI - Return of the Jedi (1983)   
19     1235                            Harold and Maude (1971)   
20     1271                        Fried Green Tomatoes (1991)   
21     1378                                  Young Guns (1988)   
22     1580                   Men in Black (a.k.a. MIB) (1997)   
23     1721                                     Titanic (1997)   
24     1884              Fear and Loathing in Las Vegas (1998)   
25     2028                         Saving Private Ryan (1998)   
26     2318                                   Happiness (1998)   
27     2513                                Pet Sematary (1989)   
28     2694                                   Big Daddy (1999)   
29     2702                               Summer of Sam (1999)   
30     2716         Ghostbusters (a.k.a. Ghost Busters) (1984)   
31     2762                            Sixth Sense, The (1999)   
32     2841                              Stir of Echoes (1999)   
33     2858                             American Beauty (1999)   
34     2959                                  Fight Club (1999)   
35     3243                                  Encino Man (1992)   
36     3510                                   Frequency (2000)   
37     3949                         Requiem for a Dream (2000)   
38     5349                                  Spider-Man (2002)   
39     5669                       Bowling for Columbine (2002)   
40     6377                                Finding Nemo (2003)   
41     7153  Lord of the Rings: The Return of the King, The...   
42     7361       Eternal Sunshine of the Spotless Mind (2004)   
43     8622                             Fahrenheit 9/11 (2004)   
44     8636                                Spider-Man 2 (2004)   
45    27369                      Daria: Is It Fall Yet? (2000)   
46    44191                              V for Vendetta (2006)   
47    48783                        Flags of Our Fathers (2006)   
48    50068                       Letters from Iwo Jima (2006)   
49    58559                            Dark Knight, The (2008)   
50    84236  White Stripes Under Great White Northern Light...   

                                             genres  
0                        Adventure|Children|Fantasy  
1                                  Action|Drama|War  
2                                       Crime|Drama  
3                                            Comedy  
4                       Comedy|Crime|Drama|Thriller  
5                                       Crime|Drama  
6                           Children|Comedy|Fantasy  
7                          Comedy|Drama|Romance|War  
8                           Action|Romance|Thriller  
9                              

### 추천 시스템에 의해서 추천된 영화목록을 보여준다 (예측평점 상위 5개)

In [124]:
get_recomm_movie(df, 3)

process time: 0.3055093288421631


Empty DataFrame
Columns: [item_id, title, genres]
Index: []